In [ ]:
import torch
import sys
sys.path.append("/home/mjfleming99/monocular-depth-estimation/")
sys.path.append("/home/justinsech/monocular-depth-estimation/")

from Dataloader import Kittiloader
from dataset import DataGenerator
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import torchvision

In [ ]:
data_path = "/home/mjfleming99/monocular-depth-estimation/dataset"
test_data = DataGenerator(data_path, phase='test', splits="eigen").create_data(1)

In [ ]:
#116 is best scene for vis
with torch.no_grad():
    for i, data in enumerate(test_data):
        if i == 116:
            a = data
            break
    
left_img = a['left_img'].cuda()

In [ ]:
model = torch.load("/home/mjfleming99/monocular-depth-estimation/repeatable_fast_epoch_3")

In [ ]:
def get_depth_map(disp, H, W):
	resized_disp = F.interpolate(disp.unsqueeze(0).unsqueeze(0), size=(H, W), mode="bilinear").squeeze()

	depth_est = SCALE_FACTOR / resized_disp

	return depth_est
def crop(depth_map):
	# We copied the crops floats for evaluation from the origonal implementation.
	# These crops are somewhat arbitrary, so coping them is the only way
	# to get a good evaluation metric.

	h, w = depth_map.shape
	return depth_map[int(0.3924324 * h):int(0.91351351 * h), int(0.0359477 * w):int(0.96405229 * w)]

def crop_img(depth_map):
	# We copied the crops floats for evaluation from the origonal implementation.
	# These crops are somewhat arbitrary, so coping them is the only way
	# to get a good evaluation metric.

	_, h, w = depth_map.shape
	return depth_map[:,int(0.3924324 * h):int(0.91351351 * h), int(0.0359477 * w):int(0.96405229 * w)]

In [ ]:
img = a['left_img']
disp1, disp2, disp3, disp4 = model.forward(left_img)
disp = disp1[:, 0, :, :].squeeze()
gt = a['depth_interp'].squeeze()

print(img.shape, disp.shape, gt.shape)

img = F.interpolate(img, size=gt.shape, mode="bilinear").squeeze()
disp = F.interpolate(disp.unsqueeze(0).unsqueeze(0), size=gt.shape, mode="bilinear").squeeze()

print(img.shape, disp.shape, gt.shape)

img = crop_img(img).detach().cpu()
disp = crop(disp).detach().cpu()
gt = crop(gt).detach().cpu()
print(img.shape, disp.shape, gt.shape)


In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
vmax = np.percentile(disp, 95)
plt.imshow(disp , cmap="magma", vmax=vmax);
plt.title("Disparity Estimate");
plt.setp(ax.get_xticklabels(), visible=False)
plt.setp(ax.get_yticklabels(), visible=False)
ax.tick_params(axis='both', which='both', length=0)

In [ ]:
SCALE_FACTOR = 5.4
MIN_DEPTH = 1e-3
MAX_DEPTH = 80
gt = a['depth_interp'].squeeze().cpu()
disp = disp1[:, 0, :, :].squeeze().cpu()
pred_depth_map = get_depth_map(disp, *gt.shape)

cropped_pred = crop(pred_depth_map)
cropped_truth = crop(gt)

cropped_truth[cropped_truth > MAX_DEPTH] = MAX_DEPTH
cropped_truth[cropped_truth < MIN_DEPTH] = MIN_DEPTH

ratio = torch.median(cropped_truth) / torch.median(cropped_pred)
cropped_scaled_pred = ratio * cropped_pred

# cropped_pred[cropped_pred > MAX_DEPTH] = MAX_DEPTH
# cropped_pred[cropped_pred < MIN_DEPTH] = MIN_DEPTH


cropped_scaled_pred[cropped_scaled_pred > MAX_DEPTH] = MAX_DEPTH
cropped_scaled_pred[cropped_scaled_pred < MIN_DEPTH] = MIN_DEPTH


cropped_truth = cropped_truth.detach().cpu()
cropped_pred =cropped_pred.detach().cpu()
cropped_scaled_pred = cropped_scaled_pred.detach().cpu()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5,10))
axes[0].hist(cropped_pred);
axes[0].set_xlabel('Depth Prediction (meters)')
axes[0].set_ylabel('Pixel Counts')
axes[0].set_title('Unscaled Depth Predictions')

axes[1].hist(cropped_scaled_pred);
axes[1].set_xlabel('Depth Prediction (meters)')
axes[1].set_ylabel('Pixel Counts')
axes[1].set_title('Median Scaled Depth Predictions')

axes[2].hist(gt.squeeze());
axes[2].set_xlabel('Depth Prediction (meters)');
axes[2].set_ylabel('Pixel Counts');
axes[2].set_title('True Depth');
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(15,10))

axes[0][0].imshow(img.permute(1,2,0));
axes[0][0].set_title("Original Left Img");

vmax = np.percentile(disp, 95)
axes[0][1].imshow(disp , cmap="magma", vmax=vmax);
axes[0][1].set_title("Disparity Estimate");

axes[1][0].imshow(cropped_scaled_pred, cmap="magma");
axes[1][0].set_title("Depth Estimate");

axes[1][1].imshow(cropped_truth, cmap="magma");
axes[1][1].set_title("True Depth");

pred = cropped_scaled_pred
target = cropped_truth

abs_rel = torch.abs(target - pred) / target
sq_rel = torch.abs(target - pred) ** 2 / target
rmse = torch.abs(target - pred)
rmse_log = torch.abs(torch.log(target) - torch.log(pred)) ** 0.5


delta = torch.max(pred/target, target/pred)
metric1 = (delta > 1.25).float()
metric2 = (delta > 1.25 ** 2).float()
metric3 = (delta > 1.25 ** 3).float()


axes[2][0].imshow(abs_rel, cmap="seismic");
axes[2][0].set_title("Abs Rel");

axes[2][1].imshow(sq_rel, cmap="seismic");
axes[2][1].set_title("Sq Rel");

axes[3][0].imshow(rmse, cmap="seismic");
axes[3][0].set_title("RMSE");

axes[3][1].imshow(rmse_log, cmap="seismic");
axes[3][1].set_title("RMSE Log");


axes[4][0].imshow(metric1, cmap="seismic");
axes[4][0].set_title("δ<1.25");

axes[4][1].imshow(metric2, cmap="seismic");
axes[4][1].set_title(f"δ<1.25\N{SUPERSCRIPT TWO}");

axes[5][0].imshow(metric3, cmap="seismic");
axes[5][0].set_title(f"δ<1.25\N{SUPERSCRIPT THREE}");

for item in axes:
    for i in item:
        plt.setp(i.get_xticklabels(), visible=False)
        plt.setp(i.get_yticklabels(), visible=False)
        i.tick_params(axis='both', which='both', length=0)


fig.delaxes(axes[5][1])
plt.tight_layout()
plt.show();